# 1d Euler Equation

**Goal:** Solve the 1d Euler's equation using a finite volume scheme.

The equation reads $\partial_t \mathbf{u} + \partial_x \mathbf{F} = 0$ with
$$
\mathbf{u} = \begin{pmatrix} \rho \\ \rho v_x \\ E \end{pmatrix} \; ,
$$
and the flux
$$
\mathbf{F} = \begin{pmatrix} \rho v_x \\ \rho v_x^2 + p \\ (E + p) v_x \end{pmatrix} \; ,
$$
where we defined the total energy $E = \rho e + \frac{1}{2} \rho v_x^2$ with $e$ the internal energy.

We use the equation of state of a polytropic ideal gas $p(e,\rho)=\rho e (\gamma - 1)$ with $\gamma = \frac{5}{3}$. 

In [51]:
using DifferentialEquations
using GLMakie

In [52]:
struct Grid
    xmin::Float64
    xmax::Float64
    Nx::Int64
    xl::Vector{Float64}
    Δx::Float64

    function Grid(; xmin=0, xmax=1, Nx=100)
        xl = range(xmin, xmax, Nx)
        Δx = xl[2] - xl[1]
        new(xmin, xmax, Nx, xl, Δx)
    end
end

In [99]:
function get_reconstruct(gd::Grid, ρl, vl, pl)
    wboundaries = (ρ=zeros(gd.Nx, 2), v=zeros(gd.Nx, 2), p=zeros(gd.Nx, 2))

    for i in 1:gd.Nx
        ip = i == gd.Nx ? 1 : i + 1

        wboundaries.ρ[i, 1] = ρl[i]
        wboundaries.ρ[i, 2] = ρl[ip]
        wboundaries.v[i, 1] = vl[i]
        wboundaries.v[i, 2] = vl[ip]
        wboundaries.p[i, 1] = pl[i]
        wboundaries.p[i, 2] = pl[ip]

        # im = i == 1 ? gd.Nx : i - 1
        # ipp = i == gd.Nx ? 2 : i == gd.Nx - 1 ? 1 : i + 2

        # κ = 1 / 3

        # wboundaries.ρ[i, 1] = ρl[i] + 1 / 4 * ((1 - κ) * (ρl[i] - ρl[im]) + (1 + κ) * (ρl[ip] - ρl[i]))
        # wboundaries.ρ[i, 2] = ρl[ip] - 1 / 4 * ((1 + κ) * (ρl[ip] - ρl[i] + (1 - κ) * (ρl[ipp] - ρl[ip])))
        # wboundaries.v[i, 1] = vl[i] + 1 / 4 * ((1 - κ) * (vl[i] - vl[im]) + (1 + κ) * (vl[ip] - vl[i]))
        # wboundaries.v[i, 2] = vl[ip] - 1 / 4 * ((1 + κ) * (vl[ip] - vl[i] + (1 - κ) * (vl[ipp] - vl[ip])))
        # wboundaries.p[i, 1] = pl[i] + 1 / 4 * ((1 - κ) * (pl[i] - pl[im]) + (1 + κ) * (pl[ip] - pl[i]))
        # wboundaries.p[i, 2] = pl[ip] - 1 / 4 * ((1 + κ) * (pl[ip] - pl[i] + (1 - κ) * (pl[ipp] - pl[ip])))
    end

    return wboundaries
end

function flux(ρ, v, p)
    e = p / (ρ * (5 / 3 - 1))

    return [
        ρ * v,
        ρ * v^2 + p,
        (ρ * e + 1 / 2 * ρ * v^2 + p) * v
    ]
end

function HLLC_solver(ρL, ρR, vL, vR, pL, pR)
    cL = sqrt(5 / 3 * pL / ρL)
    cR = sqrt(5 / 3 * pR / ρR)

    HL = pL / ρL * 5 / 3 / (5 / 3 - 1) + 1 / 2 * vL^2
    HR = pR / ρR * 5 / 3 / (5 / 3 - 1) + 1 / 2 * vR^2

    Rρ = sqrt(ρR / ρL)

    vtilde = (vL + vR * Rρ) / (1 + Rρ)
    Htilde = (HL + HR * Rρ) / (1 + Rρ)
    ctilde = sqrt((5 / 3 - 1) * (Htilde - 1 / 2 * vtilde^2))

    SL = min(vL - cL, vtilde - ctilde)
    SR = max(vR + cR, vtilde + ctilde)
    SM = (ρR * vR * (SR - vR) - ρL * vL * (SL - vL) + pL - pR) / (ρR * (SR - vR) - ρL * (SL - vL))

    if SL < 0
        return flux(ρL, vL, pL)
    elseif SL <= 0 < SM
        FL = flux(ρL, vL, pL)
        UL = [ρL, ρL * SL, EL]

        ρLstar = ρL * (SL - vL) / (SL - SM)
        pLstar = pL + ρL * (vL - SL) / (vL - SM)
        ρLstar_vLstar = (ρL * vL * (SL - vL) + pLstar - pL) / (SL - SM)

        EL = pL / (5 / 3 - 1) + 1 / 2 * ρL * vL^2
        ELstar = (EL * (SL - vL) - pL * vL + pLstar * SM) / (SL - SM)

        ULstar = [ρLstar, ρLstar_vLstar, ELstar]

        return @. FL + SL * (ULstar - UL)
    elseif SM <= 0 <= SR
        FR = flux(ρR, vR, pR)
        UR = [ρR, ρR * SR, ER]

        ρRstar = ρR * (SR - vR) / (SR - SM)
        pRstar = pR + ρR * (vR - SR) / (vR - SM)
        ρRstar_vRstar = (ρR * vR * (SR - vR) + pRstar - pR) / (SR - SM)

        ER = pR / (5 / 3 - 1) + 1 / 2 * ρR * vR^2
        ERstar = (ER * (SR - vR) - pR * vR + pRstar * SM) / (SR - SM)

        URstar = [ρRstar, ρRstar_vRstar, ERstar]

        return @. FR + SR * (URstar - UR)
    elseif SR < 0
        return flux(ρR, vR, pR)
    end
end

function get_flux(gd::Grid, ρl, vl, pl)
    Fl = zeros(gd.Nx, 3)

    wbound = get_reconstruct(gd, ρl, vl, pl)

    for i in 1:gd.Nx

        # Don't solve Riemann problem.
        Fminus = flux(wbound.ρ[i, 1], wbound.v[i, 1], wbound.p[i, 1])

        Fplus = flux(wbound.ρ[i, 2], wbound.v[i, 2], wbound.p[i, 2])

        @. Fl[i, :] = 1 / 2 * (Fplus + Fminus)
        # @. Fl[i, :] = Fplus
        # @. Fl[i, :] = Fminus

        # # Use the HLLC solver.
        # f = HLLC_solver(wbound.ρ[i, 1], wbound.ρ[i, 2], wbound.v[i, 1], wbound.v[i, 2], wbound.p[i, 1], wbound.p[i, 2])
        # @. Fl[i, :] = f
    end

    return Fl
end

function euler!(du, u, p, t)
    (; gd) = p

    @show t

    F = get_flux(gd, u[1, :], u[2, :], u[3, :])

    for i in 1:gd.Nx
        im = i == 1 ? gd.Nx : i - 1
        @. du[:, i] = -(F[i, :] - F[im, :]) / gd.Δx
    end
end

euler! (generic function with 1 method)

In [100]:
gd = Grid(xmin=0, xmax=1, Nx=500)

# ρ0l = ones(gd.Nx)
ρ0l = map(x -> 0.1 * exp(-100 * (x - 0.5)^2), gd.xl)
# v0l = zeros(gd.Nx)
v0l = map(x -> 0.1 * exp(-100 * (x - 0.5)^2), gd.xl)
p0l = ones(gd.Nx)
# p0l = map(x -> 0.001 * exp(-100 * (x - 0.5)^2), gd.xl)

# get_reconstruct(gd, ρ0l, v0l, p0l)
get_flux(gd, ρ0l, v0l, p0l);

In [101]:
u0 = zeros(3, gd.Nx)
u0[1, :] .= ρ0l
u0[2, :] .= v0l
u0[3, :] .= p0l

tspan = (0, 0.4)

prob = ODEProblem(euler!, u0, tspan, (; gd))
sol = solve(prob, Tsit5(), reltol=1e-10, abstol=1e-10, progress=true);

In [102]:
fig = Figure(size=(1200, 400))

sg = SliderGrid(fig[2, 1], (label="t", range=range(tspan[1], tspan[2], 100)))
tlift = sg.sliders[1].value

ax1 = Axis(fig[1, 1])
lines!(ax1, gd.xl, u0[1, :])
lines!(ax1, gd.xl, (@lift sol($tlift)[1, :]))

ax2 = Axis(fig[1, 2])
lines!(ax2, gd.xl, u0[2, :])
lines!(ax2, gd.xl, (@lift sol($tlift)[2, :]))

ax3 = Axis(fig[1, 3])
lines!(ax3, gd.xl, u0[3, :])
lines!(ax3, gd.xl, (@lift sol($tlift)[3, :]))

fig